In [1]:
from sklearn.model_selection import train_test_split,RepeatedKFold
from fancyimpute import MICE # imputation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve,cross_val_score,GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import RidgeCV
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

C:\Users\jjonus\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# download data and scrub

url = 'https://github.com/GinoWoz1/AdvancedHousePrices/raw/master/'
 
df_train = pd.read_csv(url + 'train.csv')
df_test = pd.read_csv(url +'test.csv')
df_train.columns.to_series().groupby(df_train.dtypes).groups

# Split data set into test and train
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
y_train = df_train.SalePrice.values
all_data = pd.concat((df_train, df_test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)


# fill in missing data

all_data['PoolQC'] = all_data['PoolQC'].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

#fill in missing data for garage, basement and mason area

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

for col in ( 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
    
for col in ('BsmtHalfBath', 'BsmtFullBath', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF1','BsmtFinSF2'):
    all_data[col] = all_data[col].fillna(0)


all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data["GarageYrBlt"] = all_data.groupby("Neighborhood")["GarageYrBlt"].transform( lambda x: x.fillna(x.mean()))

# fill in MSzoning values based on knowlege of data
    
all_data.loc[2216,'MSZoning'] = 'RM'
all_data.loc[2904,'MSZoning'] = 'RM'
all_data.loc[1915,'MSZoning'] = 'RM'
all_data.loc[2250,'MSZoning'] = 'RM'
all_data.loc[1915,'Utilities'] = 'AllPub'
all_data.loc[1945,'Utilities'] = 'AllPub'
all_data.loc[2473,'Functional'] = 'Typ'
all_data.loc[2216,'Functional'] = 'Typ'
all_data.loc[2489,'SaleType'] = 'WD'
all_data.loc[2151,'Exterior1st'] = 'Plywood'
all_data.loc[2151,'Exterior2nd'] = 'Plywood'
all_data.loc[1379,'Electrical'] = 'SBrkr'

all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

# analyze which neighborhoods have null data

all_data[pd.isna(all_data['LotFrontage'])].groupby("Neighborhood").size().sort_values(ascending = False)

# evaluation metrics and model vetting function

train = all_data[:ntrain]
test = all_data[ntrain:]
y_train = df_train.SalePrice

numeric_data_train = train.select_dtypes(include = ['float64','int64'])
solver=MICE()
Imputed_dataframe_train= pd.DataFrame(data = solver.complete(numeric_data_train),columns = numeric_data_train.columns,index = numeric_data_train.index)
numeric_data_test = test.select_dtypes(include = ['float64','int64'])
solver=MICE()
Imputed_dataframe_test= pd.DataFrame(data = solver.complete(numeric_data_test),columns = numeric_data_test.columns,index = numeric_data_test.index)

train['LotFrontage'] = Imputed_dataframe_train['LotFrontage']
test['LotFrontage'] = Imputed_dataframe_test['LotFrontage']

X_train= pd.get_dummies(train,drop_first=True)
X_test = pd.get_dummies(test,drop_first=True)

[MICE] Completing matrix with shape (1460, 36)
[MICE] Starting imputation round 1/110, elapsed time 0.001
[MICE] Starting imputation round 2/110, elapsed time 0.066
[MICE] Starting imputation round 3/110, elapsed time 0.067
[MICE] Starting imputation round 4/110, elapsed time 0.068
[MICE] Starting imputation round 5/110, elapsed time 0.069
[MICE] Starting imputation round 6/110, elapsed time 0.070
[MICE] Starting imputation round 7/110, elapsed time 0.071
[MICE] Starting imputation round 8/110, elapsed time 0.072
[MICE] Starting imputation round 9/110, elapsed time 0.073
[MICE] Starting imputation round 10/110, elapsed time 0.074
[MICE] Starting imputation round 11/110, elapsed time 0.074
[MICE] Starting imputation round 12/110, elapsed time 0.075
[MICE] Starting imputation round 13/110, elapsed time 0.076
[MICE] Starting imputation round 14/110, elapsed time 0.077
[MICE] Starting imputation round 15/110, elapsed time 0.078
[MICE] Starting imputation round 16/110, elapsed time 0.079
[M

In [3]:
# get dummies, select columns with above 0 mutual information

X_train= pd.get_dummies(train,drop_first=True)
X_test = pd.get_dummies(test,drop_first=True)

X_train = X_train[['OverallQual'	,'GarageCars'	,'TotalBsmtSF'	,'GrLivArea'	,'GarageArea'	,'FullBath'	,'YearBuilt'	,'GarageYrBlt'	,'ExterQual_TA'	,'1stFlrSF'	,'KitchenQual_TA'	,'LotFrontage'	,'ExterQual_Gd'	,'Foundation_PConc'	,'TotRmsAbvGrd'	,'KitchenQual_Gd'	,'YearRemodAdd'	,'BsmtQual_Gd'	,'Fireplaces'	,'BsmtFinSF1'	,'BsmtQual_TA'	,'FireplaceQu_None'	,'GarageFinish_Unf'	,'GarageType_Attchd'	,'BsmtUnfSF'	,'GarageType_Detchd'	,'MSSubClass_60'	,'2ndFlrSF'	,'OverallCond'	,'BsmtFinType1_GLQ'	,'Foundation_CBlock'	,'Exterior1st_VinylSd'	,'LotShape_Reg'	,'MSSubClass_30'	,'HeatingQC_TA'	,'LotArea'	,'MasVnrArea'	,'MasVnrType_None'	,'Neighborhood_NAmes'	,'MSZoning_RM'	,'CentralAir_Y'	,'SaleCondition_Partial'	,'GarageQual_None'	,'HalfBath'	,'GarageFinish_RFn'	,'FireplaceQu_Gd'	,'Neighborhood_NridgHt'	,'GarageQual_TA'	,'HouseStyle_2Story'	,'FireplaceQu_TA'	,'WoodDeckSF'	,'PavedDrive_Y'	,'GarageCond_TA'	,'Neighborhood_NoRidge'	,'MSZoning_RL'	,'SaleType_WD'	,'BedroomAbvGr'	,'Electrical_SBrkr'	,'MasVnrType_Stone'	,'Neighborhood_Somerst'	,'Neighborhood_CollgCr'	,'OpenPorchSF'	,'PoolQC_None'	,'BsmtExposure_Gd'	,'Neighborhood_Gilbert'	,'Street_Pave'	,'Neighborhood_OldTown'	,'KitchenAbvGr'	,'BsmtExposure_No'	,'YrSold'	,'Fence_MnPrv'	,'MSZoning_FV'	,'MasVnrType_BrkFace'	,'Neighborhood_MeadowV'	,'Exterior1st_Wd Sdng'	,'EnclosedPorch'	,'BsmtCond_Gd'	,'Exterior2nd_Wd Sdng'	,'KitchenQual_Fa'	,'BsmtFinType1_Rec'	,'BsmtFinSF2'	,'MSSubClass_160'	,'Exterior1st_CemntBd'	,'Neighborhood_ClearCr'	,'Neighborhood_BrkSide'	,'RoofStyle_Hip'	,'Neighborhood_IDOTRR'	,'GarageType_BuiltIn'	,'MSSubClass_50'	,'Neighborhood_StoneBr'	,'HeatingQC_Gd'	,'Neighborhood_Sawyer'	,'PavedDrive_P'	,'Fence_None'	,'HouseStyle_SFoyer'	,'GarageCond_Fa'	,'SaleCondition_Normal'	,'MSSubClass_45'	,'Functional_Typ'	,'LotConfig_Inside'	,'Exterior1st_MetalSd'	,'Electrical_FuseF'	,'Exterior1st_HdBoard'	,'BsmtQual_Fa'	,'HeatingQC_Fa'	,'Exterior2nd_Plywood'	,'ExterCond_TA'	,'BldgType_Duplex'	,'LotConfig_CulDSac'	,'Alley_None'	,'LotShape_IR2'	,'Neighborhood_Edwards'	,'BsmtFinType2_BLQ'	,'RoofStyle_Gable'	,'GarageCond_Po'	,'Functional_Min2'	,'LandSlope_Mod'	,'ScreenPorch'	,'ExterQual_Fa'	,'Neighborhood_SawyerW'	,'FireplaceQu_Po'	,'LandContour_Lvl'	,'MSSubClass_40'	,'LandContour_HLS'	,'RoofMatl_Tar&Grv'	,'SaleType_ConLw'	,'BsmtExposure_None'	,'Neighborhood_BrDale'	,'RoofStyle_Mansard'	,'MiscVal'	,'BldgType_TwnhsE'	,'Alley_Pave'	,'Exterior1st_WdShing'	,'GarageType_CarPort'	,'Fence_GdWo'	,'Exterior1st_BrkFace'	,'Condition2_Norm'	,'BsmtFinType1_Unf'	,'Heating_Grav'	,'HouseStyle_SLvl'	,'Foundation_Slab'	,'BsmtExposure_Mn'	,'MSSubClass_70'	,'BsmtFinType2_Rec'	,'Condition1_RRAe'	,'Exterior2nd_BrkFace'	,'HouseStyle_1.5Unf'	,'Neighborhood_Mitchel'	,'Electrical_FuseP'	,'Neighborhood_NWAmes'	,'MSSubClass_180'	,'HouseStyle_2.5Unf'	,'RoofStyle_Shed'	,'BsmtFinType2_GLQ'	,'LandSlope_Sev'	,'MSSubClass_85'	,'LotConfig_FR3'	,'LotConfig_FR2'	,'RoofMatl_WdShngl'	,'BsmtHalfBath'	,'Neighborhood_Blueste'	,'Exterior1st_BrkComm'	,'Neighborhood_Timber'	,'Condition1_Norm'	,'Neighborhood_SWISU'	,'BldgType_2fmCon'	,'Neighborhood_Veenker'	,'Condition1_PosA'	,'SaleCondition_Alloca'	,'SaleType_ConLD'	,'3SsnPorch'	,'LotShape_IR3'	,'Condition1_PosN'	,'BsmtFinType1_LwQ'	,'BsmtFinType1_BLQ'	,'MSZoning_RH'	,'Neighborhood_NPkVill'	,'FireplaceQu_Fa']]
X_test = X_test[['OverallQual'	,'GarageCars'	,'TotalBsmtSF'	,'GrLivArea'	,'GarageArea'	,'FullBath'	,'YearBuilt'	,'GarageYrBlt'	,'ExterQual_TA'	,'1stFlrSF'	,'KitchenQual_TA'	,'LotFrontage'	,'ExterQual_Gd'	,'Foundation_PConc'	,'TotRmsAbvGrd'	,'KitchenQual_Gd'	,'YearRemodAdd'	,'BsmtQual_Gd'	,'Fireplaces'	,'BsmtFinSF1'	,'BsmtQual_TA'	,'FireplaceQu_None'	,'GarageFinish_Unf'	,'GarageType_Attchd'	,'BsmtUnfSF'	,'GarageType_Detchd'	,'MSSubClass_60'	,'2ndFlrSF'	,'OverallCond'	,'BsmtFinType1_GLQ'	,'Foundation_CBlock'	,'Exterior1st_VinylSd'	,'LotShape_Reg'	,'MSSubClass_30'	,'HeatingQC_TA'	,'LotArea'	,'MasVnrArea'	,'MasVnrType_None'	,'Neighborhood_NAmes'	,'MSZoning_RM'	,'CentralAir_Y'	,'SaleCondition_Partial'	,'GarageQual_None'	,'HalfBath'	,'GarageFinish_RFn'	,'FireplaceQu_Gd'	,'Neighborhood_NridgHt'	,'GarageQual_TA'	,'HouseStyle_2Story'	,'FireplaceQu_TA'	,'WoodDeckSF'	,'PavedDrive_Y'	,'GarageCond_TA'	,'Neighborhood_NoRidge'	,'MSZoning_RL'	,'SaleType_WD'	,'BedroomAbvGr'	,'Electrical_SBrkr'	,'MasVnrType_Stone'	,'Neighborhood_Somerst'	,'Neighborhood_CollgCr'	,'OpenPorchSF'	,'PoolQC_None'	,'BsmtExposure_Gd'	,'Neighborhood_Gilbert'	,'Street_Pave'	,'Neighborhood_OldTown'	,'KitchenAbvGr'	,'BsmtExposure_No'	,'YrSold'	,'Fence_MnPrv'	,'MSZoning_FV'	,'MasVnrType_BrkFace'	,'Neighborhood_MeadowV'	,'Exterior1st_Wd Sdng'	,'EnclosedPorch'	,'BsmtCond_Gd'	,'Exterior2nd_Wd Sdng'	,'KitchenQual_Fa'	,'BsmtFinType1_Rec'	,'BsmtFinSF2'	,'MSSubClass_160'	,'Exterior1st_CemntBd'	,'Neighborhood_ClearCr'	,'Neighborhood_BrkSide'	,'RoofStyle_Hip'	,'Neighborhood_IDOTRR'	,'GarageType_BuiltIn'	,'MSSubClass_50'	,'Neighborhood_StoneBr'	,'HeatingQC_Gd'	,'Neighborhood_Sawyer'	,'PavedDrive_P'	,'Fence_None'	,'HouseStyle_SFoyer'	,'GarageCond_Fa'	,'SaleCondition_Normal'	,'MSSubClass_45'	,'Functional_Typ'	,'LotConfig_Inside'	,'Exterior1st_MetalSd'	,'Electrical_FuseF'	,'Exterior1st_HdBoard'	,'BsmtQual_Fa'	,'HeatingQC_Fa'	,'Exterior2nd_Plywood'	,'ExterCond_TA'	,'BldgType_Duplex'	,'LotConfig_CulDSac'	,'Alley_None'	,'LotShape_IR2'	,'Neighborhood_Edwards'	,'BsmtFinType2_BLQ'	,'RoofStyle_Gable'	,'GarageCond_Po'	,'Functional_Min2'	,'LandSlope_Mod'	,'ScreenPorch'	,'ExterQual_Fa'	,'Neighborhood_SawyerW'	,'FireplaceQu_Po'	,'LandContour_Lvl'	,'MSSubClass_40'	,'LandContour_HLS'	,'RoofMatl_Tar&Grv'	,'SaleType_ConLw'	,'BsmtExposure_None'	,'Neighborhood_BrDale'	,'RoofStyle_Mansard'	,'MiscVal'	,'BldgType_TwnhsE'	,'Alley_Pave'	,'Exterior1st_WdShing'	,'GarageType_CarPort'	,'Fence_GdWo'	,'Exterior1st_BrkFace'	,'Condition2_Norm'	,'BsmtFinType1_Unf'	,'Heating_Grav'	,'HouseStyle_SLvl'	,'Foundation_Slab'	,'BsmtExposure_Mn'	,'MSSubClass_70'	,'BsmtFinType2_Rec'	,'Condition1_RRAe'	,'Exterior2nd_BrkFace'	,'HouseStyle_1.5Unf'	,'Neighborhood_Mitchel'	,'Electrical_FuseP'	,'Neighborhood_NWAmes'	,'MSSubClass_180'	,'HouseStyle_2.5Unf'	,'RoofStyle_Shed'	,'BsmtFinType2_GLQ'	,'LandSlope_Sev'	,'MSSubClass_85'	,'LotConfig_FR3'	,'LotConfig_FR2'	,'RoofMatl_WdShngl'	,'BsmtHalfBath'	,'Neighborhood_Blueste'	,'Exterior1st_BrkComm'	,'Neighborhood_Timber'	,'Condition1_Norm'	,'Neighborhood_SWISU'	,'BldgType_2fmCon'	,'Neighborhood_Veenker'	,'Condition1_PosA'	,'SaleCondition_Alloca'	,'SaleType_ConLD'	,'3SsnPorch'	,'LotShape_IR3'	,'Condition1_PosN'	,'BsmtFinType1_LwQ'	,'BsmtFinType1_BLQ'	,'MSZoning_RH'	,'Neighborhood_NPkVill'	,'FireplaceQu_Fa']]

remove = ['BsmtQual_None','Exterior2nd_MetalSd','BsmtFinType1_None','MSSubClass_90','BsmtCond_None','BsmtEpoxsure_None','BsmtFinType2_None','GarageFinish_None','GarageCond_None','GarageType_None','MSSubClass_190','MSSubClass_80','Exterior2nd_CmentBd','SaleType_New','Exterior2nd_VinylSd']

def drop_cols(dataframe,columns_remove):
    cols =list(dataframe.columns.values)
    for col in columns_remove:
        for c in cols:
            if c == col:
                cols.remove(c)
            elif c != col:
                continue
    dataframe = dataframe[cols]
    return dataframe


X_train = drop_cols(X_train,remove)
X_test = drop_cols(X_test,remove)



In [12]:
from few import FEW
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LassoLarsCV

def rmsle_cv(y_true, y_pred) : 
    assert len(y_true) == len(y_pred)
    if not (y_true >= 0).all() and not (y_pred >= 0).all():
        raise ValueError("Mean Squared Logarithmic Error cannot be used when "
                         "targets contain negative values.")
    return -(np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_true))**2)))

learner = FEW(generations=100,population_size=500,ml=RandomForestRegressor(),scoring_function=rmsle_cv, verbosity=2)
X = X_train.values
y = y_train.values
learner.fit(X,y)

  


population size: 500
dtypes	=	None
names	=	None
normalize	=	True
lex_size	=	False
operators	=	None
weight_parents	=	True
c	=	True
otype	=	f
mdr	=	False
track_diversity	=	False
clean	=	False
classification	=	False
boolean	=	False
elitism	=	True
disable_update_check	=	False
scoring_function	=	<function rmsle_cv at 0x000000E35891FEA0>
verbosity	=	2
random_state	=	None
erc	=	False
seed_with_ml	=	True
max_stall	=	100
op_weight	=	False
fit_choice	=	None
tourn_size	=	2
sel	=	epsilon_lexicase
max_depth_init	=	2
max_depth	=	2
min_depth	=	1
ml	=	RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
crossover_rate	=	0.5
mutation_rate	=	0.5
generations	=	100
populatio







Internal CV: -0.157:   0%|                             | 0/100 [00:00<?, ?it/s]

generation 0
median fitness pop: 39039147978.39
best fitness pop: 14579773563.79
ml fitting...
current ml validation score: 0








Internal CV: -0.157:   0%|                             | 0/100 [00:00<?, ?it/s]





Internal CV: -0.157:   1%|▏                    | 1/100 [00:00<00:29,  3.33it/s]

generation 1
median fitness pop: 39038909340.11
best fitness pop: 14579773563.79
ml fitting...
current ml validation score: 0


...







Internal CV: -0.157:  98%|███████████████████▌| 98/100 [00:28<00:00,  3.45it/s]





Internal CV: -0.157:  99%|███████████████████▊| 99/100 [00:28<00:00,  3.45it/s]

generation 99
median fitness pop: 38260517220.96
best fitness pop: 7345932691.91
ml fitting...
current ml validation score: 0








Internal CV: -0.157:  99%|███████████████████▊| 99/100 [00:28<00:00,  3.45it/s]





Internal CV: -0.157: 100%|███████████████████| 100/100 [00:28<00:00,  3.45it/s]

finished. best internal val score: -0.157
final model:
 original features


FEW(boolean=False, c=True, classification=False, clean=False,
  crossover_rate=0.5, disable_update_check=False, dtypes=None,
  elitism=True, erc=False, fit_choice=None, generations=100,
  lex_size=False, max_depth=2, max_depth_init=2, max_stall=100, mdr=False,
  min_depth=1,
  ml=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
  mutation_rate=0.5, names=None, normalize=True, op_weight=False,
  operators=None, otype='f', population_size=500, random_state=None,
  scoring_function=<function rmsle_cv at 0x000000E35891FEA0>,
  seed_with_ml=True, sel='epsilon_lexicase', tourn_size=2,
  track_diversity=False, verbosity=2, weight_parents=True)